In [95]:
import json

In [96]:
with open("datasets_study.json", "r") as f:
    data = json.load(f) 

analysis_data = []


for item in data:
    if item['model']=='datasets_study.analysis':
        analysis_data.append(item)

len(analysis_data)

6777

In [97]:
problem_list = {}
for item in data:
    if item['model']=='datasets_study.problem':
       problem_list[item['pk']] = item['fields']['description']
print(problem_list)

{1: 'Spelling and grammatical errors', 2: 'Very short sentence.', 3: 'Partial or incomplete sentence.', 4: 'Self-Admitted Technical Debt (ex: TODO / FIXME / hack / XXX, etc)', 5: 'Automatically generated code/comments (ex:: // TODO auto-generated method stub).', 6: "Method name mismatches with the function/method's intent.", 7: 'Not using standard JavaDoc (Java) or docstring (Python)', 8: 'If one sample contains an example of I/O, are all the samples containing the same number of examples\r\nDifferent JavaDoc/Docstring style.\r\nGiving type hint in some samples in Python, not for all.', 9: 'URL or reference in the comment.', 12: 'Check whether the question mark is indeed asking a question about what the function does instead of being just part of the description of the code.', 13: 'Check the JavaDoc quality.', 14: 'Comment includes PII(author name, email address, etc.)', 15: 'The prompt description is unclear.', 16: 'Incorrect input/output pair example.'}


In [98]:
from collections import defaultdict
users = defaultdict(list)
analysis = defaultdict(list)
problems = defaultdict(int)


for item in analysis_data:
    if item['fields']['user_id'] == 1:
        continue
    
    users[item['fields']['user_id']].append(item['fields']['prompt'])
    analysis[item['fields']['prompt']].append(item['fields'])
    for problem in item['fields']['problems']:
        problems[problem] += 1
print(len(users))
print(len(analysis))
print(len(problems))    

2
3395
12


In [99]:
problems = sorted(problems.items(), key=lambda x: x[0])
print(problems)

[(1, 797), (2, 1), (3, 206), (4, 1), (6, 2636), (7, 872), (8, 598), (9, 36), (12, 292), (13, 2023), (15, 200), (16, 42)]


In [102]:
for item in problems:
    key = item[0]
    total = item[1]
    print(f"{key}. {problem_list[key]}: {total}")


1. Spelling and grammatical errors: 797
2. Very short sentence.: 1
3. Partial or incomplete sentence.: 206
4. Self-Admitted Technical Debt (ex: TODO / FIXME / hack / XXX, etc): 1
6. Method name mismatches with the function/method's intent.: 2636
7. Not using standard JavaDoc (Java) or docstring (Python): 872
8. If one sample contains an example of I/O, are all the samples containing the same number of examples
Different JavaDoc/Docstring style.
Giving type hint in some samples in Python, not for all.: 598
9. URL or reference in the comment.: 36
12. Check whether the question mark is indeed asking a question about what the function does instead of being just part of the description of the code.: 292
13. Check the JavaDoc quality.: 2023
15. The prompt description is unclear.: 200
16. Incorrect input/output pair example.: 42


In [ ]:
len(set(users[3])), len(set(users[4]))

(3394, 3382)

In [ ]:
extra = list(set(users[3])-set(users[4]))
extra.extend(set(users[4])-set(users[3]))

len(extra)

14

In [ ]:

with open("extra.txt", "w") as f:
    for id in extra:
        if id in users[3]:
            f.write("https://jdasilv2.pythonanywhere.com/datasets_study/prompts/{},Dristi\n".format(id))
        else:
            f.write("https://jdasilv2.pythonanywhere.com/datasets_study/prompts/{},Joy\n".format(id))



In [ ]:
mismatch = 0
for key in analysis.keys():
    current_problems = analysis[key][0]['problems']
    for item in analysis[key]:
        if item['problems'] != current_problems:
            mismatch += 1
            break
print(mismatch)

731
